In [1]:
#!/usr/bin/python3.7

In [2]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [3]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [4]:
df_raw = pd.read_csv('../data/flexible.csv')

df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [5]:
df_raw.head(10)

,p1,p2,p3,p4,p5,p6,p7,label
0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,1,0,1
3,1,1,0,0,0,0,0,1
4,1,0,0,0,0,0,1,1
5,2,0,1,0,0,0,0,1
6,2,0,0,0,0,1,0,1
7,2,1,0,0,0,0,0,1
8,2,0,0,0,0,0,1,1
9,3,0,0,1,0,0,0,1


In [6]:
df_process = df_raw
print(df_process)

         p1  p2  p3  p4  p5  p6  p7  label
0         0   0   0   0   0   0   1      1
1         0   0   0   0   0   1   0      1
2         1   0   0   0   0   1   0      1
3         1   1   0   0   0   0   0      1
4         1   0   0   0   0   0   1      1
...     ...  ..  ..  ..  ..  ..  ..    ...
19995  1455   0   1   1   1   0   1      0
19996    46   1   0   1   1   1   1      0
19997  1696   1   1   0   1   1   1      0
19998   652   1   0   0   1   1   1      0
19999  1465   1   1   1   1   1   1      0

[20000 rows x 8 columns]


In [7]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [8]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [9]:
x_train_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,label
8118,1354,0,0,0,0,0,1,1
10171,504,0,1,1,0,0,0,0
9399,1568,0,0,0,1,0,0,1
12669,93,1,1,0,0,1,1,0
18809,1670,1,0,1,0,1,1,0
13869,1824,1,1,1,1,1,1,0
3320,555,0,0,1,0,0,0,1
14689,1997,1,1,1,1,1,0,0
13087,2339,0,1,1,0,0,1,0
15992,2151,1,1,0,0,0,0,0


In [10]:
x_test_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,label
14356,2553,1,1,0,1,1,1,0
3439,574,0,1,0,0,0,0,1
12153,920,0,1,1,1,1,0,0
15029,669,0,0,1,1,0,1,0
18549,2684,0,1,1,1,0,1,0
15762,1549,1,0,0,0,0,1,0
12313,1362,1,0,1,0,0,0,0
16034,1665,1,1,1,1,1,1,0
11496,1908,1,1,0,1,0,1,0
5653,943,0,1,0,0,0,0,1


In [11]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

         p1  p2  p3  p4  p5  p6  p7
8118   1354   0   0   0   0   0   1
10171   504   0   1   1   0   0   0
9399   1568   0   0   0   1   0   0
12669    93   1   1   0   0   1   1
18809  1670   1   0   1   0   1   1
...     ...  ..  ..  ..  ..  ..  ..
13927  3171   1   0   0   0   0   1
919     154   0   1   0   0   0   0
5699    951   0   0   0   1   0   0
10742  2937   1   1   1   1   1   0
16921  3136   1   1   0   1   0   0

[16000 rows x 7 columns]
8118     1
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      0
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [12]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [13]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_process.label), len(df_process.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 7 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [14]:
print('Training XGboost model ...')

import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X, y)

print('Done')

Training XGboost model ...
Done


In [15]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       7149.0        928.0 
None

Precision:  1.0
Recall:  0.11489414386529652
Accuracy:  0.5531875
f1_score:  0.20610771793448082


# Training by bagging

In [16]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import MultinomialNB
model5 = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [17]:
model_list = [model1, model2, model3, model4, model5, model6, model7]

In [18]:
print('Training Logistics Regression model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model1,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Logistics Regression model...
Training bagging classifier...
Done!
Time: 2.4120380710810423
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7915.0          8.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9990105132962276
Recall:  1.0
Accuracy:  0.9995
f1_score:  0.9995050117559708
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2075.0          2.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9989610389610389
Recall:  1.0
Accuracy:  0.9995
f1_score:  0.9994802494802495
Training bagging classifier...
Done!
Time: 0.9844180233776569
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7915.0          8.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9990105132962276
Recall:  1.0
Accu

In [19]:
print('Training Decision Tree model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model2,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Decision Tree model...
Training bagging classifier...
Done!
Time: 0.8394348425790668
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7922.0          1.0 
           true_positive       1636.0       6441.0 
None

Precision:  0.999844768705371
Recall:  0.7974495480995419
Accuracy:  0.8976875
f1_score:  0.8872511880983539
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive        365.0       1558.0 
None

Precision:  1.0
Recall:  0.8101924076963078
Accuracy:  0.90875
f1_score:  0.8951450732548119
Training bagging classifier...
Done!
Time: 0.7981554018333554
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       1624.0       6453.0 
None

Precision:  1.0
Recall:  0.7989352482

In [20]:
print('Training MLP model directly ...')

Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model3,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training MLP model directly ...
Training bagging classifier...
Done!
Time: 6.672351898625493
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7915.0          8.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9990105132962276
Recall:  1.0
Accuracy:  0.9995
f1_score:  0.9995050117559708
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2075.0          2.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9989610389610389
Recall:  1.0
Accuracy:  0.9995
f1_score:  0.9994802494802495
Training bagging classifier...
Done!
Time: 10.29608772136271
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7913.0         10.0 
           true_positive        208.0       7869.0 
None

Precision:  0.9987308034014469
Recall:  0.974247864306054

In [21]:
print('Training SVM model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model4,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training SVM model...
Training bagging classifier...
Done!
Time: 1.7199826259166002
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       6640.0       1283.0 
           true_positive          0.0       8077.0 
None

Precision:  0.8629273504273505
Recall:  1.0
Accuracy:  0.9198125
f1_score:  0.9264208292710903
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1727.0        350.0 
           true_positive          0.0       1923.0 
None

Precision:  0.846018477782666
Recall:  1.0
Accuracy:  0.9125
f1_score:  0.9165872259294566
Training bagging classifier...
Done!
Time: 1.692418866790831
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       6046.0       1877.0 
           true_positive          0.0       8077.0 
None

Precision:  0.8114325899136026
Recall:  1.0
Accuracy:  0.8826875

In [22]:
print('Training Naive Bayesianayes model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model5,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Naive Bayesianayes model...
Training bagging classifier...
Done!
Time: 0.8935955204069614
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       5672.0       2251.0 
           true_positive       3055.0       5022.0 
None

Precision:  0.6904991062835144
Recall:  0.6217655069951715
Accuracy:  0.668375
f1_score:  0.6543322475570033
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1481.0        596.0 
           true_positive        724.0       1199.0 
None

Precision:  0.667966573816156
Recall:  0.6235049401976079
Accuracy:  0.67
f1_score:  0.6449704142011834
Training bagging classifier...
Done!
Time: 0.9058202048763633
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       5681.0       2242.0 
           true_positive       3061.0       5016.0 
None

Precision:  0.6910994

In [23]:
print('Training Random Forest model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model6,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Random Forest model...
Training bagging classifier...
Done!
Time: 8.86805647984147
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7918.0          5.0 
           true_positive        917.0       7160.0 
None

Precision:  0.9993021632937893
Recall:  0.8864677479262102
Accuracy:  0.942375
f1_score:  0.9395092507544942
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2075.0          2.0 
           true_positive        207.0       1716.0 
None

Precision:  0.9988358556461001
Recall:  0.8923556942277691
Accuracy:  0.94775
f1_score:  0.9425981873111783
Training bagging classifier...
Done!
Time: 8.64745989907533
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7918.0          5.0 
           true_positive        902.0       7175.0 
None

Precision:  0.999303621169

In [24]:
print('Training XGboost model directly ...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model7,
                             n_estimators = 50, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    

    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, model.predict(X)))
    print('Recall: ', recall_score(y_orig, model.predict(X)))
    print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
    print('f1_score: ', f1_score(y_orig, model.predict(X)))
    precision_train += precision_score(y_orig, model.predict(X))
    recall_train += recall_score(y_orig, model.predict(X))
    accuracy_train += accuracy_score(y_orig, model.predict(X))
    f1_train += f1_score(y_orig, model.predict(X))
    
    
    #test data
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, model.predict(x_test)))
    print('Recall: ', recall_score(y_test, model.predict(x_test)))
    print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
    print('f1_score: ', f1_score(y_test, model.predict(x_test)))
    
    precision_test += precision_score(y_test, model.predict(x_test))
    recall_test += recall_score(y_test, model.predict(x_test))
    accuracy_test += accuracy_score(y_test, model.predict(x_test))
    f1_test += f1_score(y_test, model.predict(x_test))
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training XGboost model directly ...
Training bagging classifier...
Done!
Time: 3.7193756261840463
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7921.0          2.0 
           true_positive         51.0       8026.0 
None

Precision:  0.9997508719481814
Recall:  0.993685774421196
Accuracy:  0.9966875
f1_score:  0.9967090965538653
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive         11.0       1912.0 
None

Precision:  1.0
Recall:  0.9942797711908476
Accuracy:  0.99725
f1_score:  0.9971316818774445
Training bagging classifier...
Done!
Time: 3.49018058180809
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7921.0          2.0 
           true_positive         46.0       8031.0 
None

Precision:  0.999751027013569
Recal